# 범주형 데이터 다루기
- 순서가 있는 범주형 데이터 / 순서가 없는 범주형 데이터


## 1. 순서가 있는 범주형 데이터 ( LabelEncoder )

### 1-(1) Rank 적 특성
- 순서 특성을 올바르게 인식(Rank)하려면 범주형의 문자열 값을 정수로 바꾸어 주어야 함 (직접 코드 작성 필요)
    - ex) size_mapping = {'XL':3, 'L':2, 'M':1}

### 1-(2) . sklearn.preprocessing -> LabelEncoder() - 인자x
- 일반 클래스 레이블 인코딩 -> LabelEncoder() : 정수로 라벨링 되기 때문에 계산시 그 정수값 자체로 해석될 수 있으니, 조심히 라벨링 해야함

#### attributes
- classes_ : class 갯수 - fit 변환까지 하고서 출력가능

#### method
- fit, transform, fit_transform(fit + transform), inverse_transform
- get_params, set_params

#### 기타
- 여러개의 열 라벨링 하고 싶으면 ColumnTransformer 더 조사

<hr>

## 2. 순서가 없는 범주형 데이터 ( One-hot-Encoding)

### 2-(1) . sklearn.preprocessing -> OneHotEncoder
- 고유한 값마다 새로운 더미 특성을 만드는 것.
- OneHotEncoder의 transform 메서드는 희소행렬을 반환한다. (sparse matrix) / 배열 내용을 보려면 희소행렬의 toarray 메서드로 일반 넘파이 배열로 변환 가능.
- **희소행렬은 대량의 데이터셋을 저장할 때 효율적이며 배열에 0이 많이 포함되어 있을때 유용함. -> 저장될 때 (0,0), (1,2), (2,1),(3,2) 식으로 1값의 위치만 저장하여 저장 공간을 아낌**
- categories='auto', drop=None, sparse=True, dtype=<class 'numpy.float64'>

### 2-(2) . Pandas -> get_dummies()
- DataFrame에 적용시 get_dummies 메서드는 문자열 열만 반환하고 나머지 열은 그대로 둠.

<hr>

- 사실상 분류면 상관없고, 회귀면 one hot? 이 낫지 않을까 싶음

## 3. 라이브러리 종류
![skl_preprocessing_img](./imgs/skl_preprocessing_img.PNG)

In [33]:
import numpy as np
import pandas as pd

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

clothes = {'price':{1:3000, 2:2000,3:1000,4:500,5:250},
               'length':{1:50, 2:25,3:15,4:10,5:5},
               'size':{1:'XL',2:'L',3:'M', 4:'S', 5:'XS'},
               'gender':{1:'m',2:'w',3:'m',4:'w',5:'h'}
               }

df = pd.DataFrame(clothes)
df

,price,length,size,gender
1,3000,50,XL,m
2,2000,25,L,w
3,1000,15,M,m
4,500,10,S,w
5,250,5,XS,h


In [34]:
# 1-(1). 순서가 있는 경우, 직접 매핑, Rank적 특성, 
size_mapping = {'XL':5,'L':4,'M':3,'S':2,'XS':1}

df_rank = df.copy()
df_rank['size'] = df_rank['size'].map(size_mapping)
df_rank

,price,length,size,gender
1,3000,50,5,m
2,2000,25,4,w
3,1000,15,3,m
4,500,10,2,w
5,250,5,1,h


In [35]:
# 1-(2). 순서가 있는 경우, LabelEncoder 
from sklearn.preprocessing import LabelEncoder

clothes_class = LabelEncoder()

# LabelEncoder를 통한 라벨링
df['size_label'] = clothes_class.fit_transform(df['size'].values)
df

# 정수 클래스 레이블을 원본 문자열로 돌리기도 가능
df['size_label_inv'] = clothes_class.inverse_transform(df['size_label'])
df

print(clothes_class.classes_)

# ==> 정수형 라벨이긴 하지만 그 정수값 자체로 해석될수 있으니, 조심히 라벨링 해야 함.

# 여러개의 열 라벨링 하고 싶으면 ColumnTransformer 더 조사

,price,length,size,gender,size_label
1,3000,50,XL,m,3
2,2000,25,L,w,0
3,1000,15,M,m,1
4,500,10,S,w,2
5,250,5,XS,h,4


,price,length,size,gender,size_label,size_label_inv
1,3000,50,XL,m,3,XL
2,2000,25,L,w,0,L
3,1000,15,M,m,1,M
4,500,10,S,w,2,S
5,250,5,XS,h,4,XS


['L' 'M' 'S' 'XL' 'XS']


In [36]:
# 2-(1). 전처리 - 순서가 없는 특성 - OneHotEncoding
from sklearn.preprocessing import OneHotEncoder

df_onehot = df.copy()

ohe = OneHotEncoder(categories='auto')
ohe.fit_transform(df_onehot)
df_onehot

<5x28 sparse matrix of type '<class 'numpy.float64'>'
	with 30 stored elements in Compressed Sparse Row format>

,price,length,size,gender,size_label,size_label_inv
1,3000,50,XL,m,3,XL
2,2000,25,L,w,0,L
3,1000,15,M,m,1,M
4,500,10,S,w,2,S
5,250,5,XS,h,4,XS


In [37]:
# 2-(2). 전처리 - 순서가 없는 특성 - getDummies()
df_dummies = df.copy()

# 변환 전
df_dummies

# 더미 변환
df_dummies = pd.get_dummies(df_dummies)
df_dummies


,price,length,size,gender,size_label,size_label_inv
1,3000,50,XL,m,3,XL
2,2000,25,L,w,0,L
3,1000,15,M,m,1,M
4,500,10,S,w,2,S
5,250,5,XS,h,4,XS


,price,length,size_label,size_L,size_M,size_S,size_XL,size_XS,gender_h,gender_m,gender_w,size_label_inv_L,size_label_inv_M,size_label_inv_S,size_label_inv_XL,size_label_inv_XS
1,3000,50,3,0,0,0,1,0,0,1,0,0,0,0,1,0
2,2000,25,0,1,0,0,0,0,0,0,1,1,0,0,0,0
3,1000,15,1,0,1,0,0,0,0,1,0,0,1,0,0,0
4,500,10,2,0,0,1,0,0,0,0,1,0,0,1,0,0
5,250,5,4,0,0,0,0,1,1,0,0,0,0,0,0,1
